In [8]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import ViltProcessor, ViltModel  # ViLT import
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import numpy as np
from PIL import Image

# Load the Excel file and check for NaN values
file_path = 'multi-sent-p.xlsx'
df = pd.read_excel(file_path)
df = df.dropna(subset=['Label_Sentiment'])  # Remove rows with NaN in 'Label_Sentiment'
df['Label_Sentiment'] = df['Label_Sentiment'].astype(int)  # Ensure Label_Sentiment is integer type

class MemeDataset(Dataset):
    def __init__(self, dataframe, img_dir, max_length=40):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.max_length = max_length
        self.processor = ViltProcessor.from_pretrained('dandelin/vilt-b32-mlm')  # ViLT processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])

        try:
            image = Image.open(img_name).convert("RGB")
            image = image.resize((224, 224))  # Resize image to 224x224
        except FileNotFoundError:
            return None

        text = self.dataframe.iloc[idx, 1]
        encoding = self.processor(images=image, text=text, return_tensors="pt", padding="max_length", 
                                  truncation=True, max_length=self.max_length)

        label = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.long)

        sample = {
            'pixel_values': encoding['pixel_values'].squeeze(),
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': label
        }
        return sample

# Define the dataset
dataset = MemeDataset(dataframe=df, img_dir='Memes/')

# Custom collate function to filter out None samples
def collate_fn(batch):
    batch = [sample for sample in batch if sample is not None]
    if len(batch) == 0:
        return None
    return torch.utils.data.dataloader.default_collate(batch)

# Define the ViLT-based Multimodal Model
class VILTHybridModel(nn.Module):
    def __init__(self):
        super(VILTHybridModel, self).__init__()
        self.vilt = ViltModel.from_pretrained('dandelin/vilt-b32-mlm')  # Load ViLT
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.vilt.config.hidden_size, 2)  # Adjusted for 2-class classification

    def forward(self, input_ids, attention_mask, pixel_values):
        outputs = self.vilt(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        pooled_output = outputs.pooler_output  # Get the pooled output
        logits = self.classifier(self.dropout(pooled_output))
        return logits

# Initialize model, loss, and optimizer
model = VILTHybridModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping parameters
patience = 2
best_loss = float('inf')
early_stop_counter = 0

# K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, test_idx) in enumerate(skf.split(dataset, df['Label_Sentiment'])):
    print(f'Fold {fold + 1}')
    
    train_subsampler = Subset(dataset, train_idx)
    test_subsampler = Subset(dataset, test_idx)
    
    train_dataloader = DataLoader(train_subsampler, batch_size=16, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_subsampler, batch_size=16, shuffle=False, collate_fn=collate_fn)
    
    # Training loop with early stopping
    model.train()
    for epoch in range(10):  
        epoch_loss = 0.0
        for batch in train_dataloader:
            if batch is None:
                continue
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], pixel_values=batch['pixel_values'])
            loss = criterion(outputs, batch['label'])
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation step for early stopping
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in test_dataloader:
                if batch is None:
                    continue
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], pixel_values=batch['pixel_values'])
                loss = criterion(outputs, batch['label'])
                val_loss += loss.item()
        
        print(f'Epoch {epoch + 1} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Check early stopping condition
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0  
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f'Early stopping triggered at epoch {epoch + 1}')
                break  
    
    # Evaluation
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in test_dataloader:
            if batch is None:
                continue
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], pixel_values=batch['pixel_values'])
            _, preds = torch.max(outputs, 1)
            all_labels.extend(batch['label'].numpy())
            all_preds.extend(preds.numpy())

    # Calculate metrics with zero_division parameter
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, zero_division=1)
    precision = precision_score(all_labels, all_preds, zero_division=1)
    recall = recall_score(all_labels, all_preds, zero_division=1)
    conf_matrix = confusion_matrix(all_labels, all_preds, labels=[0, 1])

    fold_results.append({
        'fold': fold + 1,
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    })
    print(f'Fold {fold + 1} - Accuracy: {accuracy}, F1: {f1}, Precision: {precision}, Recall: {recall}')
    print(f'Confusion Matrix:\n{conf_matrix}')

# Average results across folds
avg_accuracy = np.mean([result['accuracy'] for result in fold_results])
avg_f1 = np.mean([result['f1'] for result in fold_results])
avg_precision = np.mean([result['precision'] for result in fold_results])
avg_recall = np.mean([result['recall'] for result in fold_results])

print(f'Average Accuracy: {avg_accuracy}')
print(f'Average F1 Score: {avg_f1}')
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')


Fold 1
Epoch 1 - Train Loss: 154.9762, Val Loss: 33.4927
Epoch 2 - Train Loss: 122.7941, Val Loss: 32.8736
Epoch 3 - Train Loss: 85.8455, Val Loss: 30.8040
Epoch 4 - Train Loss: 55.4866, Val Loss: 35.5981
Epoch 5 - Train Loss: 29.2310, Val Loss: 44.6837
Early stopping triggered at epoch 5
Fold 1 - Accuracy: 0.7993827160493827, F1: 0.8108632395732299, Precision: 0.7697974217311234, Recall: 0.8565573770491803
Confusion Matrix:
[[359 125]
 [ 70 418]]
Fold 2
Epoch 1 - Train Loss: 50.3730, Val Loss: 6.6322
Epoch 2 - Train Loss: 19.7241, Val Loss: 4.5935
Epoch 3 - Train Loss: 11.5748, Val Loss: 9.0091
Epoch 4 - Train Loss: 10.7560, Val Loss: 5.6341
Early stopping triggered at epoch 4
Fold 2 - Accuracy: 0.9639546858908342, F1: 0.9634273772204807, Precision: 0.9829424307036247, Recall: 0.944672131147541
Confusion Matrix:
[[475   8]
 [ 27 461]]
Fold 3
Epoch 1 - Train Loss: 17.3279, Val Loss: 0.9848
Epoch 2 - Train Loss: 10.1570, Val Loss: 2.1144
Epoch 3 - Train Loss: 7.3223, Val Loss: 2.0232
Ea

In [10]:
# Load the new test data
test_df = pd.read_excel('test_data.xlsx')
test_df = test_df.dropna(subset=['Label_Sentiment'])  # Remove rows with NaN in 'Label_Sentiment'
test_df['Label_Sentiment'] = test_df['Label_Sentiment'].astype(int)

# Define the test dataset
test_dataset = MemeDataset(dataframe=test_df, img_dir='Memes/')

# DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# Evaluate on the test dataset
model.eval()
all_test_labels = []
all_test_preds = []

with torch.no_grad():
    for batch in test_dataloader:
        if batch is None:
            continue
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], pixel_values=batch['pixel_values'])
        _, preds = torch.max(outputs, 1)
        all_test_labels.extend(batch['label'].numpy())
        all_test_preds.extend(preds.numpy())

# Calculate evaluation metrics
test_accuracy = accuracy_score(all_test_labels, all_test_preds)
test_f1 = f1_score(all_test_labels, all_test_preds, zero_division=1)
test_precision = precision_score(all_test_labels, all_test_preds, zero_division=1)
test_recall = recall_score(all_test_labels, all_test_preds, zero_division=1)
test_conf_matrix = confusion_matrix(all_test_labels, all_test_preds, labels=[0, 1])

print(f'Test Accuracy: {test_accuracy}')
print(f'Test F1 Score: {test_f1}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Confusion Matrix:\n{test_conf_matrix}')


Test Accuracy: 0.8148148148148148
Test F1 Score: 0.7959183673469388
Test Precision: 0.8441558441558441
Test Recall: 0.752895752895753
Confusion Matrix:
[[245  36]
 [ 64 195]]
